In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
from typing import List, Tuple, Dict

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Load the dataset for recommenders

In [2]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear',
             'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[0-160],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[260-360],[3-4],False
2,3,2,WinterVacation,[2-3],Standard,[260-360],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],False
5,6,5,Easter,[4-7],Standard,[0-160],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[160-260],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[0-160],[1-1],True
8,9,7,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[0-160],[1-1],True



# Define user features based on reservations
The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [3]:
def prepare_users_df(interactions_df: pd.DataFrame):
    # display(interactions_df)
    # user_features = ['mpu_term', 'mpu_rate_plan', 'mpu_room_segment']
    # users_df = interactions_df.groupby(['user_id']).agg({
    #     'term': lambda x: x.value_counts().idxmax(),
    #     'rate_plan': lambda x: x.value_counts().idxmax(),
    #     'room_segment': lambda x: x.value_counts().idxmax(),
    # })
    # users_df.columns = user_features
    # users_df = users_df.reset_index()
    # users_df = pd.get_dummies(users_df, columns=user_features)
    # user_features = list(users_df.columns)
    # user_features.remove('user_id')

    # term_unique_values = list(interactions_df['term'].unique())
    # length_of_stay_values = list(interactions_df['length_of_stay_bucket'].unique())
    # room_segment_values = list(interactions_df['room_segment'].unique())
    # n_people_values = list(interactions_df['n_people_bucket'].unique())

    # users_df = interactions_df.groupby(['user_id'])[['term', 'n_people_bucket', 'room_segment']].agg(set)
    # users_df['term'] = users_df['term'].apply(list)
    # users_df['n_people_bucket'] = users_df['n_people_bucket'].apply(list)
    # users_df['room_segment'] = users_df['room_segment'].apply(list)
    # display(users_df)

    one_hot = pd.get_dummies(interactions_df, columns=['term', 'rate_plan', 'room_segment'])
    one_hot = one_hot.drop(columns=['n_people_bucket', 'length_of_stay_bucket', 'weekend_stay'])
    options = {}
    for col in one_hot.columns:
        if col == 'user_id' or col == 'item_id':
            continue
        else:
            options[col] = 'sum'
    one_hot = one_hot.groupby(['user_id']).agg(options)
    one_hot = one_hot / one_hot.sum(axis=1).values.reshape(-1, 1)
    one_hot: pd.DataFrame = one_hot.rename_axis(None, axis=1).fillna(0)
    users_df = one_hot.add_prefix('user_')
    user_features = list(users_df.columns)
    users_df = users_df.reset_index()
    return users_df, user_features


users_df, user_features = prepare_users_df(interactions_df)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

,user_id,user_term_WinterVacation,user_term_Easter,user_term_OffSeason,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_Christmas,user_rate_plan_Standard,user_rate_plan_Nonref,user_room_segment_[0-160],user_room_segment_[160-260],user_room_segment_[260-360],user_room_segment_[360-500],user_room_segment_[500-900]
0,1,0.030303,0.0,0.227273,0.030303,0.045455,0.000000,0.000000,0.0,0.166667,0.166667,0.318182,0.000000,0.000000,0.015152,0.0
40,50,0.014493,0.0,0.144928,0.101449,0.072464,0.000000,0.000000,0.0,0.086957,0.246377,0.246377,0.014493,0.043478,0.028986,0.0
84,96,0.030303,0.0,0.227273,0.045455,0.015152,0.015152,0.000000,0.0,0.090909,0.242424,0.181818,0.045455,0.060606,0.045455,0.0
102,115,0.000000,0.0,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.000000,0.166667,0.166667,0.000000,0.000000,0.0
371,706,0.031746,0.0,0.170635,0.063492,0.047619,0.015873,0.003968,0.0,0.325397,0.007937,0.079365,0.103175,0.099206,0.051587,0.0
1383,1736,0.011494,0.0,0.160920,0.068966,0.091954,0.000000,0.000000,0.0,0.057471,0.275862,0.264368,0.000000,0.011494,0.057471,0.0
7301,7779,0.000000,0.0,0.166667,0.000000,0.166667,0.000000,0.000000,0.0,0.333333,0.000000,0.083333,0.000000,0.250000,0.000000,0.0


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br>
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [4]:
def prepare_items_df(interactions_df):
    # item_features = ['mpi_term', 'mpi_rate_plan', 'mpi_room_segment']
    # items_df = interactions_df.groupby(['item_id']).agg({
    #     'term': lambda x: x.value_counts().idxmax(),
    #     'rate_plan': lambda x: x.value_counts().idxmax(),
    #     'room_segment': lambda x: x.value_counts().idxmax(),
    # })
    # items_df.columns = item_features
    # items_df = items_df.reset_index()
    # items_df = pd.get_dummies(items_df, columns=item_features)
    # item_features = list(items_df.columns)
    # item_features.remove('item_id')

    one_hot = pd.get_dummies(interactions_df, columns=['term', 'rate_plan', 'room_segment'])
    one_hot = one_hot.drop(columns=['n_people_bucket', 'length_of_stay_bucket', 'weekend_stay'])
    options = {}
    for col in one_hot.columns:
        if col == 'user_id' or col == 'item_id':
            continue
        else:
            options[col] = 'sum'
    one_hot = one_hot.groupby(['item_id']).agg(options)
    one_hot = one_hot / one_hot.sum(axis=1).values.reshape(-1, 1)
    one_hot = one_hot.rename_axis(None, axis=1).fillna(0)
    items_df = one_hot.add_prefix('item_')
    item_features = list(items_df.columns)
    items_df = items_df.reset_index()

    return items_df, item_features


items_df, item_features = prepare_items_df(interactions_df)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

,item_id,item_term_WinterVacation,item_term_Easter,item_term_OffSeason,item_term_HighSeason,item_term_LowSeason,item_term_MayLongWeekend,item_term_NewYear,item_term_Christmas,item_rate_plan_Standard,item_rate_plan_Nonref,item_room_segment_[0-160],item_room_segment_[160-260],item_room_segment_[260-360],item_room_segment_[360-500],item_room_segment_[500-900]
0,0,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.0
1,1,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
2,2,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.333333,0.0,0.0
3,3,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.333333,0.000000,0.0,0.0
4,4,0.333333,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.0
5,5,0.000000,0.333333,0.000000,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.333333,0.000000,0.000000,0.0,0.0
6,6,0.000000,0.000000,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.000000,0.333333,0.000000,0.0,0.0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        super().__init__()
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos

        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None

        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)

    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.

        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """

        interactions_df = interactions_df.copy()

        # Prepare users_df and items_df
        users_df, user_features = prepare_users_df(interactions_df)

        self.users_df = users_df
        self.user_features = user_features

        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]

        # Generate negative interactions
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]

        interactions_df.loc[:, 'interacted'] = 1

        negative_interactions = []

        ########################
        ########################
        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.

        user_ids = list(interactions_df['user_id'])
        unique_item_ids = list(interactions_df['item_id'].unique())
        self.rng.shuffle(unique_item_ids)
        unique_item_ids_len = len(unique_item_ids)

        copy = interactions_df.copy()
        copy['val'] = 1
        interactions_dict = copy.set_index(['user_id', 'item_id']).val.to_dict()

        for user_id in user_ids:
            added = 0
            while added < self.n_neg_per_pos:
                random_item_id = unique_item_ids[self.rng.randint(0, unique_item_ids_len)]
                if (user_id, random_item_id) not in interactions_dict:
                    interactions_dict[(user_id, random_item_id)] = 0
                    negative_interactions.append((user_id, random_item_id, 0))
                    added += 1

        ########################
        ########################
        ########################

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])

        # Get the input data for the model
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        # display(interactions_df)
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)

    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """

        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]

        ########################
        ########################
        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.

        # Score the items
        users_df = users_df.copy()
        users_df = pd.merge(users_df, self.users_df, on='user_id', how='left')
        users_df = users_df.fillna(0)
        items_df, item_features = prepare_items_df(items_df)

        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])

        for ix, user in users_df.iterrows():

            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            user_df = user.to_frame().T
            input_df = pd.merge(user_df, items_df, how="cross")
            input_df_copy = input_df.drop(columns=['user_id', 'item_id'])
            # display(input_df)
            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            scores = self.model.predict(input_df_copy.values)

            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = np.argsort(-scores)[:n_recommendations]

            ########################
            ########################
            ########################
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': input_df.iloc[item_id]['item_id'],
                        'score': scores[item_id]
                    }
                )

            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df


class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()


class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)


class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)


class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """

    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)

# Quick test of the recommender

In [6]:
# display(interactions_df.loc[interactions_df['item_id'] == 2258])

items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()

In [7]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

In [8]:
# Recommender method
recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']),
                                                     interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,623.0,0.804356,OffSeason,[8-inf],Standard,[0-160],[3-4],True
1,1.0,20.0,0.804356,OffSeason,[2-3],Standard,[0-160],[5-inf],False
2,1.0,209.0,0.804356,OffSeason,[4-7],Standard,[0-160],[5-inf],False
3,1.0,24.0,0.804356,OffSeason,[4-7],Standard,[0-160],[2-2],False
4,1.0,206.0,0.804356,OffSeason,[4-7],Standard,[0-160],[5-inf],True
5,1.0,26.0,0.804356,OffSeason,[2-3],Standard,[0-160],[3-4],True
6,1.0,49.0,0.804356,OffSeason,[4-7],Standard,[0-160],[1-1],True
7,1.0,42.0,0.804356,OffSeason,[2-3],Standard,[0-160],[1-1],False
8,1.0,46.0,0.804356,OffSeason,[4-7],Standard,[0-160],[3-4],True
9,1.0,32.0,0.804356,OffSeason,[2-3],Standard,[0-160],[2-2],False


# Tuning method

In [9]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [10]:
from hyperopt import hp, fmin, tpe, Trials
import traceback


def tune_recommender(recommender_class, interactions_df, items_df,
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1

    if not succeded:
        return None

    # Validate

    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results,
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5',
                                    'NDCG@10'])

    display(results)

    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [11]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|██████████| 10/10 [14:54<00:00, 89.44s/trial, best loss: -0.06982649174777825]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.001358,0.00611,0.007807,0.040394,0.001358,0.004267,0.004998,0.015528


Best parameters:
{'n_neg_per_pos': 5.0}


In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

In [ ]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [237]:
# cb_user_item_recommender = LinearRegressionCBUIRecommender(
#     **{'n_neg_per_pos': 7})  # Initialize your recommender here with the best params from tuning
cb_user_item_recommender = LinearRegressionCBUIRecommender(best_param_set)
# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.0,0.000339,0.003734,0.005092,0.0,0.000214,0.001631,0.00206


In [238]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results,
    columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.013578,0.040054,0.062797,0.120163,0.013578,0.028683,0.037912,0.056361


In [239]:
tts_results = pd.concat([linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.000000,0.000339,0.003734,0.005092,0.000000,0.000214,0.001631,0.002060
1,AmazonRecommender,0.013578,0.040054,0.062797,0.120163,0.013578,0.028683,0.037912,0.056361
